# Edmonton Food Assets Project

this notebook uses the module `gspread` to connect with a google spreadsheet and manipulate it.


another joe change

** about gspread **

I got it from doing this (in the terminal): 
> ```conda install -c bryanwweber gspread```


_**To Do**_

Current unresolved issue is that in the list of descriptions that are in a column of the spreadsheet there are sometimes two URLs.
We only want the the URL-encoded one. the first one has bad encoding of the characters.

it would also be cool to use this modules https://github.com/tuomur/python-odata to directly access edmonton's open data


this is a radical change that should only be in the joe branch

In [18]:
import pandas as pd
import gspread
from oauth2client.service_account import ServiceAccountCredentials

Got my start using this [Twilio blog](https://www.twilio.com/blog/2017/02/an-easy-way-to-read-and-write-to-a-google-spreadsheet-in-python.html)


- Go to the Google APIs Console.
- Create a new project.
- Click Enable API. Search for and enable the Google Drive API.
- Create credentials for a Web Server to access Application Data.
- Name the service account and grant it a Project Role of Editor.
- Download the JSON file.
- Copy the JSON file to your code directory and rename it to client_secret.json


 

There is one last required step to authorize your app, and it’s easy to miss!

Find the  client_email inside client_secret.json. Back in your spreadsheet, click the Share button in the top right, and paste the client email into the People field to give it edit rights. Hit Send.

In [19]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials


# use creds to create a client to interact with the Google Drive API
scope = ['https://spreadsheets.google.com/feeds']
creds = ServiceAccountCredentials.from_json_keyfile_name('../client_secret.json', scope)
client = gspread.authorize(creds)

# # Find a workbook by name and open the first sheet
# # Make sure you use the right name here.
# sheet = client.open("Copy of Open YEGFood Asset Map").Beehives

# # Extract and print all of the values
# list_of_hashes = sheet.get_all_records()
# print(list_of_hashes)

In [20]:
g_sheets = client.open("Copy of Open YEGFood Asset Map")

In [21]:
g_sheets.worksheets()

[<Worksheet 'Edible City Trees' id:od6>,
 <Worksheet 'Vacant Lots For Urban Agriculture' id:ooj9lq6>,
 <Worksheet 'Beehives' id:oab072s>,
 <Worksheet 'Community Gardens' id:opqt86e>]

# google to sheet to DataFrame

In [22]:
bees_sheet = g_sheets.get_worksheet(2)
_d = bees_sheet.get_all_records(); _d[:10]

[{'': '',
  'Name': '2816 39 Ave NW (Existing)',
  'category': 'Hive Removal',
  'description': '',
  'lat': 53.4757,
  'long': -113.383},
 {'': '',
  'Name': 'Radisson Hotel (Poisoned)',
  'category': 'Hive Removal',
  'description': 'Radisson Hotel Edmonton South. Manager called because they had a swarm of bees on the building. However, it turns out that they were living in the wall and sprayed (by an exterminator) the day before.',
  'lat': 53.48253,
  'long': -113.494},
 {'': '',
  'Name': '9736 87 Ave NW (Existing)',
  'category': 'Hive Removal',
  'description': "Can you help us remove bees and honey from our chimney? We live in Old Strathcona, 100 year old house, non-functioning fireplace. I think I know the point of entry. There was a lot of activity a few weeks ago, now it's quiet but I know they are there. We want them gone for good but are most concerned about getting the honey out of the walls and chimney.",
  'lat': 53.52294,
  'long': -113.481},
 {'': '',
  'Name': '12846

In [23]:
beehives_df = pd.DataFrame.from_dict(_d)

In [24]:
beehives_df.head()

,,Name,category,description,lat,long
0,,2816 39 Ave NW (Existing),Hive Removal,,53.4757,-113.383
1,,Radisson Hotel (Poisoned),Hive Removal,Radisson Hotel Edmonton South. Manager called ...,53.4825,-113.494
2,,9736 87 Ave NW (Existing),Hive Removal,Can you help us remove bees and honey from our...,53.5229,-113.481
3,,12846 134 St NW (Existing),Hive Removal,,53.5873,-113.552
4,,74 Wize Ct NW (Unsure),Hive Removal,"June 4, 2015. Swarm that recently moved under ...",53.4981,-113.599


In [25]:
beehives_df.columns

Index(['', 'Name', 'category', 'description', 'lat', 'long'], dtype='object')

# DataFrame to Google Sheet?

In [26]:
garden_sheet = g_sheets.get_worksheet(3)

In [27]:
import re
url_re = re.compile(r"""((?i)\b((?:[a-z][\w-]+:(?:/{1,3}|[a-z0-9%])|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’])))""")



In [39]:
import urllib.parse 
for u in garden_df.description.str.extract(url_re)[0][:10]:
    print(urllib.parse.quote(u))

http%3A//sustainablefoodedmonton.org/alex-taylor-garden/
http%3A//sustainablefoodedmonton.org/boyle-street-community-garden/
http%3A//sustainablefoodedmonton.org/city-of-edmonton-plant-a-row-grow-a-row/
http%3A//sustainablefoodedmonton.org/our-urban-eden/
http%3A//sustainablefoodedmonton.org/oasis-at-the-bay/
http%3A//sustainablefoodedmonton.org/st-andrews-community-garden/
http%3A//edmcommunitygardens.blogspot.com/search/label/Trinity%2520Manor
http%3A//sustainablefoodedmonton.org/victory-rooftop-gardens/
http%3A//sustainablefoodedmonton.org/dickinsfield-blooms-community-garden/
http%3A//sustainablefoodedmonton.org/elmwood-park/


/Users/mannalytics/anaconda/envs/geopandas_test_env/lib/python3.5/site-packages/ipykernel/__main__.py:2: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  from ipykernel import kernelapp as app


Add url column to sheet

In [29]:
garden_df = pd.DataFrame.from_dict(garden_sheet.get_all_records())
garden_df.loc[:,'url'] = garden_df.description.str.extract(url_re)[0]


/Users/mannalytics/anaconda/envs/geopandas_test_env/lib/python3.5/site-packages/ipykernel/__main__.py:2: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  from ipykernel import kernelapp as app


In [31]:
garden_df.head()

,Name,URLs,description,lat,long,url
0,Alex Taylor Community Garden,http://sustainablefoodedmonton.org/alex-taylor...,9321 Jasper Avenue<br><br>http://sustainablefo...,53.54686,-113.477,http://sustainablefoodedmonton.org/alex-taylor...
1,Boyle Street,http://sustainablefoodedmonton.org/boyle-stree...,10235 â€“ 96 street http://sustainablefoodedmo...,53.55165,-113.482,http://sustainablefoodedmonton.org/boyle-stree...
2,City of Edmonton Plant-a-Row Grow-a-Row,http://sustainablefoodedmonton.org/city-of-edm...,13111 Meridian Street<br><br> <br>Website:<br>...,53.58996,-113.345,http://sustainablefoodedmonton.org/city-of-edm...
3,Our Urban Eden,http://sustainablefoodedmonton.org/our-urban-e...,Address:<br>Bellamy Hill Road & 99 Avenue<br><...,53.53724,-113.496,http://sustainablefoodedmonton.org/our-urban-e...
4,Oasis at the Bay,http://sustainablefoodedmonton.org/oasis-at-th...,10826 113 Street<br><br><br>Website:<br>http:/...,53.55336,-113.516,http://sustainablefoodedmonton.org/oasis-at-th...


How do I find the last row in a sheet? `row_count` just gives all the rows without regard for data.

In [32]:
garden_sheet.row_count

1000

In [38]:
for i in garden_df.ix[50:,'url']:
    print(i)

http://sustainablefoodedmonton.org/beverly-allianâ€¦mmunity-garden/
http://sustainablefoodedmonton.org/clareview-community-ecogarden/
http://sustainablefoodedmonton.org/kensington-community-garden/
http://sustainablefoodedmonton.org/eastwood-community-garden/
http://sustainablefoodedmonton.org/our-garden/
http://sustainablefoodedmonton.org/rio-gardens/
http://sustainablefoodedmonton.org/grow-forward-grovenor/
http://sustainablefoodedmonton.org/canora/
http://sustainablefoodedmonton.org/holy-spirit-lutheran-church/
http://sustainablefoodedmonton.org/chappelle/
http://sustainablefoodedmonton.org/twin-brooks/
http://sustainablefoodedmonton.org/trinity-lutheran-church/
http://sustainablefoodedmonton.org/ellerslie-garden/
http://sustainablefoodedmonton.org/garrison-hortiâ€¦r-base-members/
http://sustainablefoodedmonton.org/st-albert-pariâ€¦eritage-garden/


this is how I actually update the spreadsheet using `update_cell` 

In [15]:
num_rows = len(garden_df)
offset = 2
url_col_num = 5
garden_sheet.update_cell(1,url_col_num,'URLs')
column = 5
for r in range(offset,num_rows + offset):
    garden_sheet.update_cell(r,url_col_num, garden_df.ix[r-offset,'url'])
    